In [24]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import multiprocessing

from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import cross_val_score

from scipy.misc import imread, imresize
import imageio
import skimage
from sklearn.model_selection import cross_val_score

In [2]:
!pwd

/Users/Mao/Documents/fall_mod2_18/dl/project/dl_distracted_driver


In [3]:
! ls /Users/Mao/Documents/fall_mod2_18/dl/project/data/imgs

test  train


In [6]:
os.getcwd()

'/Users/Mao/Documents/fall_mod2_18/dl/project/dl_distracted_driver'

In [9]:
labels = 'c0,c1,c2,c3,c4,c5,c6,c7,c8,c9'.split(',')
def get_train():
    "Get the training data into a DataFrame"
    path = '/Users/Mao/Documents/fall_mod2_18/dl/project/'
    root = os.path.join(path, 'data', 'imgs')
    labels = [i for i in os.listdir(os.path.join(root,'train')) if not i.startswith(".")]
    labels.sort()
    print('Found labels: ', *labels)
    data = []
    for lab in labels:
        paths = os.listdir(os.path.join(root,'train', lab))
        X = [(os.path.join(root, 'train', lab, i), lab) for i in paths]
        data.extend(X)
    import random
    random.shuffle(data) # since labels were sorted
    df = pd.DataFrame({'paths': [i[0] for i in data],
                       'target': [i[1] for i in data]})

    for cl in labels:
        df[cl] = df.target == cl
    df.drop('target', 1, inplace=True)
    return df

In [10]:
train = get_train().sample(2000)

Found labels:  c0 c1 c2 c3 c4 c5 c6 c7 c8 c9


In [11]:
train.head

<bound method NDFrame.head of                                                    paths     c0     c1     c2  \
14186  /Users/Mao/Documents/fall_mod2_18/dl/project/d...  False   True  False   
8220   /Users/Mao/Documents/fall_mod2_18/dl/project/d...  False  False   True   
9857   /Users/Mao/Documents/fall_mod2_18/dl/project/d...  False   True  False   
10534  /Users/Mao/Documents/fall_mod2_18/dl/project/d...   True  False  False   
20700  /Users/Mao/Documents/fall_mod2_18/dl/project/d...  False  False  False   
5760   /Users/Mao/Documents/fall_mod2_18/dl/project/d...  False   True  False   
21940  /Users/Mao/Documents/fall_mod2_18/dl/project/d...  False  False  False   
996    /Users/Mao/Documents/fall_mod2_18/dl/project/d...  False   True  False   
4112   /Users/Mao/Documents/fall_mod2_18/dl/project/d...  False  False  False   
21584  /Users/Mao/Documents/fall_mod2_18/dl/project/d...  False  False  False   
1812   /Users/Mao/Documents/fall_mod2_18/dl/project/d...   True  False  False  

In [12]:
def _apply_df(args):
    df, func, kwargs = args
    return df.apply(func, **kwargs)

def apply_by_multiprocessing(df, func, **kwargs):
    workers = multiprocessing.cpu_count()
    print("Using", workers, "workers to apply function")
    pool = multiprocessing.Pool(processes=workers)
    result = pool.map(_apply_df, [(d, func, kwargs)
            for d in np.array_split(df, workers)])
    pool.close()
    return pd.concat(list(result))

def getimage(x):
    return imresize(imread(x, 'L'), (100, 100)).flatten()

In [13]:
train['images'] = apply_by_multiprocessing(train.paths, getimage) 
X = np.array([i for i in train.images])
print('Loaded training data')

Using 4 workers to apply function


/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`

Loaded training data


In [18]:
train.head()

,paths,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9,images
14186,/Users/Mao/Documents/fall_mod2_18/dl/project/d...,False,True,False,False,False,False,False,False,False,False,"[61, 62, 67, 71, 76, 87, 96, 99, 105, 105, 99,..."
8220,/Users/Mao/Documents/fall_mod2_18/dl/project/d...,False,False,True,False,False,False,False,False,False,False,"[60, 65, 75, 89, 108, 122, 118, 98, 89, 63, 53..."
9857,/Users/Mao/Documents/fall_mod2_18/dl/project/d...,False,True,False,False,False,False,False,False,False,False,"[23, 22, 22, 23, 23, 23, 24, 23, 23, 24, 24, 2..."
10534,/Users/Mao/Documents/fall_mod2_18/dl/project/d...,True,False,False,False,False,False,False,False,False,False,"[71, 77, 78, 80, 83, 88, 97, 103, 107, 111, 11..."
20700,/Users/Mao/Documents/fall_mod2_18/dl/project/d...,False,False,False,False,False,False,False,False,True,False,"[39, 37, 37, 37, 38, 39, 38, 37, 37, 37, 36, 3..."


### Train Classifier

In [26]:
classifiers = [RandomForestClassifier(n_jobs=-1,
                                      n_estimators=100
                                      ) for i in labels]
targets = [train[i] for i in labels]


In [27]:
for i, clf, Y in zip(labels, classifiers, targets):
    print(cross_val_score(clf, X, Y, scoring='log_loss'))
    clf.fit(X, Y)
    print('Trained', i)
print('Done training')

/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


[-0.17848698 -0.16157476 -0.1541172 ]
Trained c0


/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


[-0.15846179 -0.13223316 -0.14525962]
Trained c1


/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


...

[-0.0784613  -0.13589578 -0.08375682]
Trained c7


/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


[-0.13223728 -0.16220584 -0.10850109]
Trained c8


/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)
/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/metrics/scorer.py:137: DeprecationWarning: Scoring method log_loss was renamed to neg_log_loss in version 0.18 and will be removed in 0.20.
  sample_weight=sample_weight)


[-0.13637749 -0.1401746  -0.14253608]
Trained c9
Done training


In [19]:
def get_test():
    "Get test data into a DataFrame"
    path = '/Users/Mao/Documents/fall_mod2_18/dl/project/'
    root = os.path.join(path, 'data', 'imgs')
    paths = os.listdir(os.path.join(root, 'test'))
    x = [os.path.join(root, 'test', i) for i in paths]
    x.sort()
    df = pd.DataFrame({'paths': x,
                       })
    return df

In [20]:
print('loading images')
test = get_test()

loading images


In [22]:
test['paths'][0]

'/Users/Mao/Documents/fall_mod2_18/dl/project/data/imgs/test/img_1.jpg'

#### Prediciton

In [19]:
test = get_test()
test['images'] = apply_by_multiprocessing(test.paths, getimage)
X = np.array([i for i in test.images])
print('Done loading test data.')
print("Running predictions")
results = []
for index, clf in enumerate(classifiers):
    results.append(clf.predict_proba(X)[:,1])
    print('c'+str(index), 'Done with prediction')

print('Creating submission')
c0, c1, c2, c3, c4, c5, c6, c7, c8, c9 = results
sub = pd.DataFrame({'img': test.paths.apply(lambda x:x.split('/')[-1]),
                    'c0': c0,'c1': c1,'c2': c2,
                    'c3': c3,'c4': c4,'c5': c5,
                    'c6': c6,'c7': c7,'c8': c8,
                    'c9': c9,})
sub[['img', 'c0', 'c1', 'c2', 'c3',
     'c4', 'c5', 'c6', 'c7', 'c8', 'c9']].to_csv('submission.csv', index=False)
print('Done!')


Using 4 workers to apply function


/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  from ipykernel import kernelapp as app
/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:15: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`

Done loading test data.
Running predictions
c0 Done with prediction
c1 Done with prediction
c2 Done with prediction
c3 Done with prediction
c4 Done with prediction
c5 Done with prediction
c6 Done with prediction
c7 Done with prediction
c8 Done with prediction
c9 Done with prediction
Creating submission
Done!
